In [1]:
import os
from pathlib import Path
import sys

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [2]:
import pandas as pd
import tidytcells as tt

In [3]:
data_dir = Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data")

### Load and clean data

In [4]:
raw = pd.read_csv(data_dir/"raw"/"gdb"/"gdb_20230606.csv")

In [8]:
raw_human = raw[raw["meta:tcr_species"] == "HomoSapiens"].copy()

In [9]:
for tr_col in ("TRAV", "TRAJ", "TRBV", "TRBJ"):
    raw_human[tr_col] = raw_human[tr_col].map(lambda x: None if type(x) != str else tt.tcr.standardize(x, enforce_functional=True))
    raw_human[tr_col] = raw_human[tr_col].map(lambda x: None if type(x) != str or not tr_col in x else x)

for cdr3_col in ("CDR3A", "CDR3B"):
    raw_human[cdr3_col] = raw_human[cdr3_col].map(lambda x: None if type(x) != str else tt.junction.standardize(x, strict=True))

/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV15" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV15".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV28" for species homosapiens: gene has no functional alleles. Attempted fix: "TRAV28".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV8-5*01" for species homosapiens: nonfunctional allele. Attempted fix: "TRAV8-5*01".
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib/python3.11/site-packages/tidytcells/_utils/warnings.py:12: UserWarning: Failed to standardize "TRAV33*01" for species homosapiens: nonfunctional allele. Attempted fix: "TRAV33*01".
  warn(
/home/yutanagano/Projects/tcr_embedder

### Focus on beta chains

In [10]:
beta_raw_human = raw_human.drop(columns=["TRAV", "CDR3A", "TRAJ"])
beta_raw_human = beta_raw_human.dropna(subset=["TRBV", "Epitope"])

In [11]:
beta_raw_human = beta_raw_human.groupby("Epitope").filter(lambda x: len(x) > 1)

In [9]:
epitope_counts = beta_raw_human.Epitope.value_counts()

In [10]:
hold_out_epitopes = epitope_counts[epitope_counts > 200][-20:].index.to_list()

In [11]:
brh_holdout = beta_raw_human[beta_raw_human.Epitope.isin(hold_out_epitopes)]
brh_train = beta_raw_human[~beta_raw_human.index.isin(brh_holdout.index)]

In [12]:
brh_holdout = brh_holdout.groupby("Epitope").sample(n=200, random_state=420)

In [13]:
top_20_epitopes = brh_train.Epitope.value_counts()[:20].index

In [14]:
brh_valid = brh_train[brh_train.Epitope.isin(top_20_epitopes)].groupby("Epitope").sample(frac=0.1)

In [15]:
brh_train = brh_train[~brh_train.index.isin(brh_valid.index)]

### Save data

In [16]:
brh_train.to_csv(data_dir/"preprocessed"/"gdb"/"train.csv", index=False)
brh_valid.to_csv(data_dir/"preprocessed"/"gdb"/"valid.csv", index=False)
brh_holdout.to_csv(data_dir/"preprocessed"/"gdb"/"test.csv", index=False)

In [13]:
beta_raw_human.to_csv(data_dir/"preprocessed"/"gdb"/"beta_raw_human.csv", index=False)